Código para web crawling da Biblioteca Digital Brasiliana Guita e José Mindlin

Este código implementa um web crawler que acessa a Biblioteca Digital Brasiliana Guita e José Mindlin, pertencente à Universidade de São Paulo (USP). Ele utiliza a plataforma DSpace para gerenciar seu acervo digital. O script foi desenvolvido em Python e usa as bibliotecas requests_html e requests para realizar requisições HTTP, renderizar o conteúdo dinâmico das páginas e baixar arquivos PDF.
Principais Funcionalidades:

    1- Coleta de informações do documento: O código navega pelas páginas dos documentos na Biblioteca Brasiliana usando IDs sequenciais atribuídos a cada obra. Ele extrai dados essenciais, como o nome do autor, título da obra e data de publicação.

    2- Geração de nomes de arquivos: O nome do arquivo PDF é gerado a partir de uma combinação de autor, título (truncado para um limite de 63 caracteres) e a data de publicação. O formato do nome facilita a organização dos arquivos baixados e evita nomes muito longos.

    3- Download automático de PDFs: Após acessar a página de um documento, o script identifica o botão de download do PDF e faz o download do arquivo correspondente. O processo inclui uma verificação básica para garantir que o arquivo foi baixado corretamente (salvando-o na pasta configurada).

    4- Tratamento de erros: O código é robusto para lidar com possíveis problemas, como documentos não encontrados ou links inválidos. Ele garante que o processo continue mesmo se ocorrerem erros, como uma página inexistente ou falha na conexão.

Detalhes Técnicos:

    Bibliotecas:
        requests_html: Usada para renderizar o HTML dinâmico das páginas, o que é necessário para capturar botões de download que podem ser carregados dinamicamente.
        requests: Lida diretamente com o download dos arquivos PDF após o conteúdo ser identificado.

    Sessões: O script utiliza HTMLSession, que permite gerenciar cookies e manter uma sessão contínua ao acessar diferentes páginas do site, simulando um comportamento de navegador.

    Tratamento de exceções: O código é capaz de capturar exceções comuns, como a tentativa de acessar uma página inexistente (erro 404) ou falhas ao baixar arquivos. Isso permite que ele continue executando o processo de download para outros documentos, mesmo que ocorra uma falha em algum ponto.

    Estrutura de pastas: Os arquivos baixados são salvos na pasta ./usp/. Se essa pasta não existir, é necessário criá-la previamente ou modificar o código para que ele crie a pasta automaticamente.

Instalação de requisitos

In [ ]:
pip install requests requests-html


In [ ]:
from requests_html import HTMLSession  # Biblioteca para gerenciar sessões HTML e renderizar páginas dinâmicas
import requests  # Biblioteca para realizar requisições HTTP
import warnings  # Biblioteca para gerenciar warnings
warnings.filterwarnings("ignore")  # Ignora avisos de warnings que podem surgir durante a execução

# Definindo o User-Agent para simular um navegador comum
DEFAULT_USER_AGENT = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36'

session = HTMLSession()  # Inicializa uma sessão HTML para gerenciar requisições contínuas
headers = {'User-Agent': DEFAULT_USER_AGENT}  # Define o cabeçalho da requisição HTTP com o User-Agent

# Definindo URLs principais da Biblioteca Digital da USP
DOMAIN = 'https://digital.bbm.usp.br/handle/bbm/'  # URL base para acessar os documentos
SHORT_DOMAIN = 'https://digital.bbm.usp.br'  # Domínio base para concatenar com URLs internas
MAX_LENGTH = 63  # Limite máximo de caracteres para o nome do arquivo gerado


# Função para tratar o nome do autor, retorna o primeiro e segundo nome concatenados
def parse_author(author):
    list = author.split(" ")  # Divide o nome completo do autor em partes
    #return author
    return list[0] + list[1]  # Retorna a junção do primeiro e segundo nome do autor


# Função para gerar o nome do arquivo PDF baseado no autor, título e data de emissão
def generate_file_name(rawAuthor, title, issued_date):
    author = parse_author(rawAuthor)  # Pega o nome tratado do autor
    maxTitleLength = MAX_LENGTH - 6 - len(author)  # Calcula o espaço disponível para o título no nome do arquivo
    partialTitle = title[0:maxTitleLength].replace(":", "_")  # Trunca o título e substitui ":" por "_"
    file_name = author + '@' + partialTitle + '$' + str(issued_date)  # Formata o nome do arquivo como "autor@titulo$data"
    return file_name  # Retorna o nome do arquivo gerado


# Função para pegar a URL do PDF na página
def get_pdf_url(response):
    buttons = response.html.find('.btn-primary')  # Procura os botões de download com a classe 'btn-primary'
    pdf_url = SHORT_DOMAIN + buttons[2].attrs.get('href')  # Pega o terceiro botão e obtém o link do PDF
    return pdf_url  # Retorna a URL do PDF


# Função para fazer o download do PDF e salvar no diretório './usp/'
def download_pdf(url, file_name):
    response = requests.get(url, headers=headers, verify=False)  # Faz a requisição HTTP para baixar o PDF
    # Salva o arquivo PDF no diretório './usp/' com o nome gerado
    with open('./usp/' + file_name + '.pdf', 'wb') as w:  # Abre o arquivo no modo binário
        w.write(response.content)  # Escreve o conteúdo do PDF no arquivo


# Função principal para baixar as imagens (PDFs) usando o ID do livro
def download_image(bookId):
    url = DOMAIN + str(bookId)  # Constrói a URL do livro a partir do ID
    response = session.get(url, headers=headers, verify=False)  # Faz uma requisição GET à página do livro
    response.html.render()  # Renderiza a página para processar o conteúdo dinâmico

    # Extrai o autor, título e data de emissão da página
    author = response.html.find('.dc_contributor_author')  # Encontra o nome do autor na página
    title = response.html.find('.dc_title')  # Encontra o título do documento na página
    issued_date = response.html.find('.dc_date_issued')  # Encontra a data de emissão do documento

    # Gera o nome do arquivo usando os dados extraídos
    file_name = generate_file_name(author[1].text, title[1].text, issued_date[1].text)  
    pdf_url = get_pdf_url(response)  # Obtém a URL do PDF para download
    download_pdf(pdf_url, file_name)  # Faz o download do PDF e o salva com o nome gerado

    # Exibe uma mensagem indicando que o download foi concluído com sucesso
    print("================================== DOWNLOAD COMPLETO ==================================\n\n")


# Loop para baixar documentos com IDs de 1 a 8200
for id in range(1, 10):  # Itera pelos IDs de 1 a 8200 (altere conforme necessário. 8200 é o tamanho total do acervo quando mensurado em 2022)
    print(f"\n==================================== BAIXANDO {str(id)} ====================================")
    try:
        download_image(id)  # Tenta baixar o documento correspondente ao ID atual
    except Exception as e:  # Captura qualquer erro que possa ocorrer durante o processo
        print(f'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX ID {str(id)} NÃO EXISTE! XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX\n\n')
        pass  # Ignora o erro e continua o loop para o próximo ID
